## 1. Loader

In [2]:
import torchvision
import torchvision.transforms as transforms

# CenterCrop : 이미지 가운데 부분만 잘라서 사용하겠다
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
# 이미지 전처리를 위한 transforms.Compose

trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, 
                         batch_size=4, #batchsize 4개 지정해서 12500개의 데이터를 불러오겠다 
                         shuffle=True, 
                         drop_last=False)

testloader = DataLoader(testset, 
                        batch_size=4,
                        shuffle=False,
                        drop_last=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
trainiter = iter(trainloader) #한 batch만 뽑아서 확인해보기 
images, labels = trainiter.next() #batch size 4 로 지정한 shape확인 가능 

print(len(trainloader))
print(images.shape)
print(labels) #1차원 tensor인데 length 가 4인! 

12500
torch.Size([4, 3, 32, 32])
tensor([5, 8, 8, 4])


## 2-1. Model - ReLU

In [13]:
src = {'input_size':3*32*32,   #config #환경설정 #들어가야하는 input들 한번에 묶어놓은 것 
       'hidden_size1':256,
       'hidden_size2':128,
       'hidden_size3':64,
       'output_size':10,
       'init_weight_range':0.5,
       'num_epochs':5,
       'batch_size':4,
       'learning_rate':1e-3}

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class CifarModel_Yum(nn.Module):
    def __init__(self, src):
        super(CifarModel_Yum, self).__init__()
        self.fc1 = nn.Linear(src['input_size'], src['hidden_size1']) #인자:input값의 개수 , output값의 개수  #층을 하나 만드는 것!
        self.fc2 = nn.Linear(src['hidden_size1'], src['hidden_size2'])
        self.fc3 = nn.Linear(src['hidden_size2'], src['hidden_size3'])
        self.fc4 = nn.Linear(src['hidden_size3'], src['output_size']) #최종 length가 10인 tensor가 나옴
                             
        ## sequential layer
        self.seq_fc = nn.Sequential(
                            nn.Linear(src['input_size'], src['hidden_size1']),
                            nn.Linear(src['hidden_size1'], src['hidden_size2']),
                            nn.Linear(src['hidden_size2'], src['hidden_size3']),
                            nn.Linear(src['hidden_size3'], src['output_size'])
                            )   #하나의 layer로 묶음  #위랑 결과 같음!
        
        self.init_range = src['init_weight_range']
        
    def init_weight(self):
        self.fc1.weight.data.uniform_(-self.init_range, self.init_range) #가중치 초기화 
        self.fc2.weight.data.uniform_(-self.init_range, self.init_range)
        self.fc3.weight.data.uniform_(-self.init_range, self.init_range)
        self.fc4.weight.data.uniform_(-self.init_range, self.init_range)
        
        for fc in self.seq_fc:
            fc.weight.data.uniform_(-self.init_range, self.init_range)
        
    def forward(self, img): #4 layers 
        x = img.view(img.shape[0], -1) #image shape 4*3*32*32 중 4차원만 유지하고 나머지는 한 차원으로 묶어버리겠다
        #--------------------        
        x = F.relu(self.fc1(x)) #activation function으로 ReLU 사용 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        y = self.fc4(x)
        #--------------------
        
        ## sequencial로 써도 위와 같은 모델 
        #  y = self.seq_fc(x)

        return y #마지막 y는 length10짜리 tensor

In [9]:
model = CifarModel_Yum(src) #init하는 부분 
y = model(images) #forward부분 

In [10]:
print(y.shape) 

torch.Size([4, 10])


## 3. Train / Eval

### 손실 함수 및 optimizer 정의

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()  #loss
optimizer = optim.SGD(model.parameters(), #gradient SGD
                      src['learning_rate'], 
                      momentum=0.9)

### 네트워크 학습
단순히 데이터를 반복시켜 네트워크와 optimizer의 입력으로 넘겨주기만 하면 된다.

In [14]:
from tqdm import tqdm  

model.init_weight()

for epoch in tqdm(range(src['num_epochs'])):   #epoch 5번 
    current_loss = 0.0
#     model.train(True)
    
    for i, data in enumerate(trainloader): #4개씩 묶여있는 사진들이 루프 한번만에!
        # get the inputs
        inputs, labels = data  #image에 사진이랑 클래스 있음 
        
        # zero the parameter gradients
        optimizer.zero_grad()  #0으로 초기화 
        
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels) #실제 사진과 확률의 차이
        loss.backward()  #편미분 값들 계산
        optimizer.step()  #계산한 값들로 update
        
        # print statistics
        step = i + 1
        current_loss += loss.item()
        
        if step % 1000 == 0 and step != 0:     # print every 1000 mini-batches
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %
                  (epoch + 1, src['num_epochs'], step, len(trainloader)//1000 * 1000, current_loss / 1000)) #누적합의 평균 
            current_loss = 0.0  


  0%|                                                    | 0/5 [00:00<?, ?it/s]

Epoch [1/5], Step [1000/12000], Loss: 4.4639
Epoch [1/5], Step [2000/12000], Loss: 2.3055
Epoch [1/5], Step [3000/12000], Loss: 2.3030
Epoch [1/5], Step [4000/12000], Loss: 2.3082
Epoch [1/5], Step [5000/12000], Loss: 2.3050
Epoch [1/5], Step [6000/12000], Loss: 2.3029
Epoch [1/5], Step [7000/12000], Loss: 2.3034
Epoch [1/5], Step [8000/12000], Loss: 2.3031
Epoch [1/5], Step [9000/12000], Loss: 2.3031
Epoch [1/5], Step [10000/12000], Loss: 2.3032
Epoch [1/5], Step [11000/12000], Loss: 2.3028
Epoch [1/5], Step [12000/12000], Loss: 2.3017



 20%|████████                                | 1/5 [16:42<1:06:50, 1002.74s/it]

Epoch [2/5], Step [1000/12000], Loss: 2.3033
Epoch [2/5], Step [2000/12000], Loss: 2.3045
Epoch [2/5], Step [3000/12000], Loss: 2.3028
Epoch [2/5], Step [4000/12000], Loss: 2.3036
Epoch [2/5], Step [5000/12000], Loss: 2.3031
Epoch [2/5], Step [6000/12000], Loss: 2.3035
Epoch [2/5], Step [7000/12000], Loss: 2.3024
Epoch [2/5], Step [8000/12000], Loss: 2.3037
Epoch [2/5], Step [9000/12000], Loss: 2.3031
Epoch [2/5], Step [10000/12000], Loss: 2.3034
Epoch [2/5], Step [11000/12000], Loss: 2.3032
Epoch [2/5], Step [12000/12000], Loss: 2.3037



 40%|████████████████▊                         | 2/5 [38:56<55:05, 1101.94s/it]

...


 60%|████████████████████████                | 3/5 [1:03:38<40:31, 1216.00s/it]

Epoch [4/5], Step [1000/12000], Loss: 2.3034
Epoch [4/5], Step [2000/12000], Loss: 2.3034
Epoch [4/5], Step [3000/12000], Loss: 2.3025
Epoch [4/5], Step [4000/12000], Loss: 2.3038
Epoch [4/5], Step [5000/12000], Loss: 2.3033
Epoch [4/5], Step [6000/12000], Loss: 2.3034
Epoch [4/5], Step [7000/12000], Loss: 2.3027
Epoch [4/5], Step [8000/12000], Loss: 2.3038
Epoch [4/5], Step [9000/12000], Loss: 2.3028
Epoch [4/5], Step [10000/12000], Loss: 2.3028
Epoch [4/5], Step [11000/12000], Loss: 2.3030
Epoch [4/5], Step [12000/12000], Loss: 2.3030



 80%|████████████████████████████████        | 4/5 [1:28:35<21:40, 1300.35s/it]

Epoch [5/5], Step [1000/12000], Loss: 2.3035
Epoch [5/5], Step [2000/12000], Loss: 2.3029
Epoch [5/5], Step [3000/12000], Loss: 2.3024
Epoch [5/5], Step [4000/12000], Loss: 2.3036
Epoch [5/5], Step [5000/12000], Loss: 2.3031
Epoch [5/5], Step [6000/12000], Loss: 2.3031
Epoch [5/5], Step [7000/12000], Loss: 2.3035
Epoch [5/5], Step [8000/12000], Loss: 2.3034
Epoch [5/5], Step [9000/12000], Loss: 2.3029
Epoch [5/5], Step [10000/12000], Loss: 2.3034
Epoch [5/5], Step [11000/12000], Loss: 2.3023
Epoch [5/5], Step [12000/12000], Loss: 2.3032



100%|████████████████████████████████████████| 5/5 [1:53:20<00:00, 1355.63s/it]

### 평가 데이터를 이용한 네트워크 평가
학습 데이터셋을 이용해 총 5번 반복하면서 학습을 시켰다. 그러나 실제로 네트워크가 무엇인가를 배웠는지에 대하여 테스트를 해야한다.

뉴럴 네트워크의 출력인 클래스 label을 예측하고 실제 데이터와 비교함으로써 테스트를 수행할 수 있는데 만약 예측 결과가 올바르다면 올바른 예측 리스트에 샘플을 추가할 수 있다.

In [16]:
import torch
# Test the Model
correct = 0
total = 0
for i, data in enumerate(testloader):
    inputs, labels = data
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.shape[0]
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total)) 

Accuracy of the network on the 2500 test images: 10 %


성능이 너무 낮아서 다른 시도를 해보기로 한당,,, 

# 2-2. Model - Sigmoid

In [21]:
class CifarModel_sigmoid(nn.Module):
    def __init__(self, src):
        super(CifarModel_sigmoid, self).__init__()
        self.fc1 = nn.Linear(src['input_size'], src['hidden_size1']) #인자:input값의 개수 , output값의 개수  #층을 하나 만드는 것!
        self.fc2 = nn.Linear(src['hidden_size1'], src['hidden_size2'])
        self.fc3 = nn.Linear(src['hidden_size2'], src['hidden_size3'])
        self.fc4 = nn.Linear(src['hidden_size3'], src['output_size']) #최종 length가 10인 tensor가 나옴
                             
        ## sequential layer
        self.seq_fc = nn.Sequential(
                            nn.Linear(src['input_size'], src['hidden_size1']),
                            nn.Linear(src['hidden_size1'], src['hidden_size2']),
                            nn.Linear(src['hidden_size2'], src['hidden_size3']),
                            nn.Linear(src['hidden_size3'], src['output_size'])
                            )   #하나의 layer로 묶음  #위랑 결과 같음!
        
        self.init_range = src['init_weight_range']
        
    def init_weight(self):
        self.fc1.weight.data.uniform_(-self.init_range, self.init_range) #가중치 초기화 
        self.fc2.weight.data.uniform_(-self.init_range, self.init_range)
        self.fc3.weight.data.uniform_(-self.init_range, self.init_range)
        self.fc4.weight.data.uniform_(-self.init_range, self.init_range)
        
        for fc in self.seq_fc:
            fc.weight.data.uniform_(-self.init_range, self.init_range)
        
    def forward(self, img): #4 layers 
        x = img.view(img.shape[0], -1) #image shape 4*3*32*32 중 4차원만 유지하고 나머지는 한 차원으로 묶어버리겠다
        #--------------------  
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x)) #activation function으로 sigmoid 사용 
        x = torch.sigmoid(self.fc3(x))
        y = self.fc4(x)
        #--------------------
        
        ## sequencial로 써도 위와 같은 모델 
        #  y = self.seq_fc(x)

        return y

In [22]:
model = CifarModel_sigmoid(src) #init하는 부분 #준비물들 준비 
y = model(images) #forward부분

## 네트워크 학습

In [23]:
from tqdm import tqdm 

model.init_weight()

for epoch in tqdm(range(src['num_epochs'])):    
    current_loss = 0.0
#     model.train(True)
    
    for i, data in enumerate(trainloader): 
        # get the inputs
        inputs, labels = data   
        
        # zero the parameter gradients
        optimizer.zero_grad()  
        
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step()  
        
    
        step = i + 1
        current_loss += loss.item()
        
        if step % 1000 == 0 and step != 0:    
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %
                  (epoch + 1, src['num_epochs'], step, len(trainloader)//1000 * 1000, current_loss / 1000))  
            current_loss = 0.0 


  0%|                                                    | 0/5 [00:00<?, ?it/s]

Epoch [1/5], Step [1000/12000], Loss: 3.1298
Epoch [1/5], Step [2000/12000], Loss: 3.1061
Epoch [1/5], Step [3000/12000], Loss: 3.1346
Epoch [1/5], Step [4000/12000], Loss: 3.1063
Epoch [1/5], Step [5000/12000], Loss: 3.1490
Epoch [1/5], Step [6000/12000], Loss: 3.1297
Epoch [1/5], Step [7000/12000], Loss: 3.1167
Epoch [1/5], Step [8000/12000], Loss: 3.1246
Epoch [1/5], Step [9000/12000], Loss: 3.1337
Epoch [1/5], Step [10000/12000], Loss: 3.1651
Epoch [1/5], Step [11000/12000], Loss: 3.1408
Epoch [1/5], Step [12000/12000], Loss: 3.1074



 20%|████████                                | 1/5 [25:35<1:42:20, 1535.21s/it]

Epoch [2/5], Step [1000/12000], Loss: 3.0936
Epoch [2/5], Step [2000/12000], Loss: 3.1109
Epoch [2/5], Step [3000/12000], Loss: 3.1167
Epoch [2/5], Step [4000/12000], Loss: 3.1375
Epoch [2/5], Step [5000/12000], Loss: 3.1531
Epoch [2/5], Step [6000/12000], Loss: 3.1224
Epoch [2/5], Step [7000/12000], Loss: 3.1282
Epoch [2/5], Step [8000/12000], Loss: 3.0965
Epoch [2/5], Step [9000/12000], Loss: 3.1335
Epoch [2/5], Step [10000/12000], Loss: 3.1369
Epoch [2/5], Step [11000/12000], Loss: 3.1636
Epoch [2/5], Step [12000/12000], Loss: 3.1261



 40%|████████████████                        | 2/5 [52:00<1:17:30, 1550.20s/it]

...


 60%|████████████████████████                | 3/5 [1:17:54<51:43, 1551.50s/it]

Epoch [4/5], Step [1000/12000], Loss: 3.1559
Epoch [4/5], Step [2000/12000], Loss: 3.1205
Epoch [4/5], Step [3000/12000], Loss: 3.1083
Epoch [4/5], Step [4000/12000], Loss: 3.1581
Epoch [4/5], Step [5000/12000], Loss: 3.1188
Epoch [4/5], Step [6000/12000], Loss: 3.1118
Epoch [4/5], Step [7000/12000], Loss: 3.1114
Epoch [4/5], Step [8000/12000], Loss: 3.1395
Epoch [4/5], Step [9000/12000], Loss: 3.1264
Epoch [4/5], Step [10000/12000], Loss: 3.1334
Epoch [4/5], Step [11000/12000], Loss: 3.0924
Epoch [4/5], Step [12000/12000], Loss: 3.0989



 80%|████████████████████████████████        | 4/5 [1:43:45<25:51, 1551.22s/it]

Epoch [5/5], Step [1000/12000], Loss: 3.0721
Epoch [5/5], Step [2000/12000], Loss: 3.1033
Epoch [5/5], Step [3000/12000], Loss: 3.1274
Epoch [5/5], Step [4000/12000], Loss: 3.1523
Epoch [5/5], Step [5000/12000], Loss: 3.1514
Epoch [5/5], Step [6000/12000], Loss: 3.1017
Epoch [5/5], Step [7000/12000], Loss: 3.1119
Epoch [5/5], Step [8000/12000], Loss: 3.1556
Epoch [5/5], Step [9000/12000], Loss: 3.1230
Epoch [5/5], Step [10000/12000], Loss: 3.1572
Epoch [5/5], Step [11000/12000], Loss: 3.1192
Epoch [5/5], Step [12000/12000], Loss: 3.1399



100%|████████████████████████████████████████| 5/5 [2:09:47<00:00, 1554.60s/it]

## 평가 데이터를 이용한 네트워크 평가

In [24]:
import torch
# Test the Model
correct = 0
total = 0
for i, data in enumerate(testloader):
    inputs, labels = data
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.shape[0]
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total)) 

Accuracy of the network on the 2500 test images: 9 %
